## Iteractively refine dataset labels

In [45]:
import os
import pathlib
import typing as t
import random
import requests
import json as json_package
import collections
import glob

import datasets
import tokenizers
import ipywidgets
import colorama
import tqdm

import segmentador
from config import *
import interactive_labeling


%load_ext autoreload
%autoreload 2


CN = colorama.Fore.RED
CT = colorama.Fore.YELLOW
CLN = colorama.Fore.CYAN
CSR = colorama.Style.RESET_ALL


VOCAB_SIZE = 6000
AUTOSAVE_IN_N_INSTANCES = 20


DATASET_SPLIT = "test"
CACHED_INDEX_FILENAME = "refined_indices.csv"
BRUTE_DATASET_DIR = "../data"

TARGET_DATASET_NAME = f"df_tokenized_split_0_120000_{VOCAB_SIZE}"

REFINED_DATASET_DIR = os.path.join(BRUTE_DATASET_DIR, "refined_datasets", TARGET_DATASET_NAME)

BRUTE_DATASET_URI = os.path.join(BRUTE_DATASET_DIR, TARGET_DATASET_NAME)
CACHED_INDEX_URI = os.path.join(REFINED_DATASET_DIR, CACHED_INDEX_FILENAME)


assert BRUTE_DATASET_URI != REFINED_DATASET_DIR


logit_model = segmentador.BERTSegmenter(
    uri_model=f"../pretrained_segmenter_model/4_{VOCAB_SIZE}_layer_model",
    device="cpu",
)

pbar_dump = tqdm.auto.tqdm(desc="Instances until next dump:", total=AUTOSAVE_IN_N_INSTANCES)
it_counter = 0

random.seed(17)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Instances until next dump::   0%|          | 0/20 [00:00<?, ?it/s]

Instances until next dump::  65%|█████████████████████████████████████████████████████████████████████████████████▎                                           | 13/20 [07:32<04:03, 34.81s/it]


In [2]:
tokenizer = tokenizers.Tokenizer.from_file(f"../tokenizers/{VOCAB_SIZE}_subwords/tokenizer.json")
tokenizer.get_vocab_size()

6000

## Load dataset

In [3]:
try:    
    with open(CACHED_INDEX_URI, "r") as f_index:
        cached_indices = set(map(int, f_index.read().split(",")))
        
    print(f"Loaded {len(cached_indices)} indices from disk.")

except FileNotFoundError:
    cached_indices = set()
    
    
new_refined_instances = collections.defaultdict(list)
df_brute = datasets.load_from_disk(BRUTE_DATASET_URI)
df_brute

Loaded 111 indices from disk.


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 114884
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14361
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14361
    })
})

## Iteractive refinement

In [11]:
df_split = df_brute[DATASET_SPLIT]

fn_rand = lambda: random.randint(0, df_split.num_rows)

df_view_labels = df_split["labels"]
df_view_input_ids = df_split["input_ids"]

cls2id = {
    "no-op": 0,
    "seg": 1,
    "n-start": 2,
    "n-end": 3,
}

def print_labels(input_ids, labels, input_is_tokens: bool = False):
    seg_counter = 1
    
    print(end=CSR)
    print(end=f"{CLN}{seg_counter}.{CSR} ")
    
    if not input_is_tokens:
        tokens = list(map(tokenizer.id_to_token, input_ids))
    
    else:
        tokens = input_ids
    
    for i, (tok, lab) in enumerate(zip(tokens, labels)):
        if lab == cls2id["seg"]:
            seg_counter += 1
            print("\n\n", end=f"{CLN}{seg_counter}.{CSR} ")
        
        if lab == cls2id["n-start"]:
            print(end=CN)
            
        if lab == cls2id["n-end"]:
            print(end=CSR)
            
        print(tok, end=" ")
        
    return tokens


def dump_refined_dataset():
    new_subset = datasets.Dataset.from_dict(new_refined_instances, split=DATASET_SPLIT)
    shard_id = len(glob.glob(os.path.join(REFINED_DATASET_DIR, f"{DATASET_SPLIT}_*")))
    REFINED_DATASET_SHARD_URI = os.path.join(REFINED_DATASET_DIR, f"{DATASET_SPLIT}_{shard_id}")
    new_subset.save_to_disk(REFINED_DATASET_SHARD_URI)
    new_refined_instances.clear()
    
    with open(CACHED_INDEX_URI, "w") as f_index:
        f_index.write(",".join(map(str, sorted(cached_indices))))
    
    it_counter = 0
    
    print(f"Saved progress in '{REFINED_DATASET_SHARD_URI}'.")

In [329]:
id_ = fn_rand()
while id_ in cached_indices:
    id_ = fn_rand()


input_ids = df_view_input_ids[id_]
labels = df_view_labels[id_]
tokens = print_labels(input_ids, labels)

1. [CLS] CÂMARA DOS DEPUTADOS 

2. PROJETO DE LEI Nº , DE 2013 ( Do Sr . M ##Á ##R ##CIO F ##RAN ##ÇA ) 

3. Acrescenta dispositivos à Lei nº 10 . 74 ##1 , de 1º de outubro de 2003 , que dispõe sobre o Estatuto do I ##dos ##o e dá outras providências , para criar a Cu ##rado ##ria dos I ##dos ##os . 

4. O Congresso Nacional decreta : 

5. Art . 1º Esta Lei acrescenta dispositivos à Lei nº 10 . 74 ##1 , de 1º de outubro de 2003 , que dispõe sobre o Estatuto do I ##dos ##o e dá outras providências , para criar a Cu ##rado ##ria dos I ##dos ##os . 

6. Art . 2º A Lei nº 10 . 74 ##1 , de 1º de outubro de 2003 , passa a vigorar acrescida dos seguintes artigos : 

7. Capítulo VII Da Cu ##rado ##ria dos I ##dos ##os 

8. Art . 68 - A . A Cu ##rado ##ria dos I ##dos ##os é órgão permanente e aut ##ônom ##o , não juris ##dic ##ion ##al , encar ##reg ##ado pela sociedade de z ##ela ##r pelo cumprimento dos direitos idosos , definidos nesta Lei . 

9. Art . 68 - B . Em cada Município e em cada R

In [316]:
logits = logit_model(df_split[id_], return_logits=True).logits
interactive_labeling.open_example(tokens, labels, logits=logits)
lock_save = True

In [317]:
ret = interactive_labeling.retrieve_refined_example()
labels = ret["labels"]
print_labels(tokens, labels, input_is_tokens=True)
lock_save = False

1. [CLS] Of ##ício nº 83 ##7 ( SF ) 

2. Brasília , em 5 de junho de 2014 . 

3. A Sua Excelência o Senhor Deputado Mar ##cio B ##it ##tar Primeiro - Secretário da Câmara dos Deputados 

4. Ass ##unto : Emenda do Senado a Projeto de Lei da Câmara . 

5. Senhor Primeiro - Secretário , Comun ##ico a Vossa Excelência que o Senado Federal aprov ##ou , em revisão e com emenda , o Projeto de Lei da Câmara nº 6 , de 2014 ( PL nº 2 . 201 , de 2011 , nessa Casa ) , que “ Institui a grat ##ificação por exercício cum ##ula ##tivo de ofício ##s dos membros do Ministério Público da União e dá outras providências ” . Em anexo , encaminh ##o a Vossa Excelência o aut ##óg ##ra ##f ##o referente à emenda em apre ##ço . 

6. At ##enci ##osamente , m ##m / pl ##c ##1 ##4 - 04 ##1 

7. Emenda ##s do Senado ao Projeto de Lei da Câmara nº 6 , de 2014 ( PL nº 2 . 201 , de 2011 , na Casa de origem ) , que 

8. “ Institui a grat ##ificação por exercício cum ##ula ##tivo de ofício ##s dos membros do Ministério 

In [330]:
if not lock_save and id_ not in cached_indices:
    it_counter += 1
    cached_indices.add(id_)

    for key, val in df_split[id_].items():
        if key != "labels":
            new_refined_instances[key].append(val.copy())

        else:
            new_refined_instances[key].append(labels)
     
    pbar_dump.update()


if it_counter % AUTOSAVE_IN_N_INSTANCES == 0:
    dump_refined_dataset()
    pbar_dump.reset()
    
    
print(pbar_dump)

Saved progress in '../data/refined_datasets/df_tokenized_split_0_120000_6000/test_7'.
Instances until next dump::   0%|          | 0/20 [00:00<?, ?it/s]
